In [ ]:
from dataclasses import dataclass, asdict
from typing import Literal, Self

@dataclass
class Expr:
    conj: Literal["And", "Or"]
    items: list[Self | int]

# Helper to make test creation cleaner
def And(*args: Expr | int): return Expr("And", list(args))
def Or(*args: Expr | int): return Expr("Or", list(args))

@dataclass
class TestCase:
    id: str
    input: Expr
    expected: list[list[int]]
    desc: str


In [29]:

# --- The Comprehensive Test Suite ---

# 1. ATOMIC & BASE CASES
# Testing that primitives survive or are wrapped correctly
atomic_tests = [
    TestCase(
        "atomic_and",
        And(1),
        [[1]],
        "Single atom inside AND"
    ),
    TestCase(
        "atomic_or",
        Or(1),
        [[1]],
        "Single atom inside OR"
    ),
    TestCase(
        "simple_and",
        And(1, 2, 3),
        [[1, 2, 3]],
        "Flat AND list"
    ),
    TestCase(
        "base_simple_or",
        Or(1, 2, 3),
        [[1], [2], [3]],
        "Flat OR list"
    ),
    TestCase(
        "deep_atomic_and",
        And(And(And(1))),
        [[1]],
        "Single atom inside 3-layer AND"
    ),
    TestCase(
        "deep_atomic_or",
        Or(Or(Or(1))),
        [[1]],
        "Single atom inside 3-layer OR"
    ),
    TestCase(
        "deep_atomic_mixed",
        And(Or(And(Or(1)))),
        [[1]],
        "Single atom inside 4-layer alternating conjunctions"
    )
]

# 2. ASSOCIATIVITY (Flattening)
# Logic: And(1, And(2, 3)) == And(1, 2, 3)
# Logic: Or(1, Or(2, 3)) == Or(1, 2, 3)
associativity_tests = [
    TestCase(
        "assoc_nested_and",
        And(1, And(2, 3), And(4)),
        [[1, 2, 3, 4]],
        "Nested ANDs should merge into one list"
    ),
    TestCase(
        "assoc_nested_or",
        Or(1, Or(2, 3), Or(4)),
        [[1], [2], [3], [4]],
        "Nested ORs should merge into distinct lists"
    ),
    TestCase(
        "assoc_mixed_depth",
        And(1, And(2, And(3, 4))),
        [[1, 2, 3, 4]],
        "Deeply nested ANDs"
    ),
]

# 3. DISTRIBUTIVITY (The Hard Part)
# Logic: And(A, Or(B, C)) -> Or(And(A, B), And(A, C))
distributivity_tests = [
    TestCase(
        "dist_simple",
        And(1, Or(2, 3)),
        [[1, 2], [1, 3]],
        "Distribute atom 1 over (2 OR 3)"
    ),
    TestCase(
        "dist_right_side",
        And(Or(1, 2), 3),
        [[1, 3], [2, 3]],
        "Distribute atom 3 over (1 OR 2)"
    ),
    TestCase(
        "dist_cartesian_product",
        And(Or(1, 2), Or(3, 4)),
        [[1, 3], [1, 4], [2, 3], [2, 4]],
        "Full Cartesian product: (1|2) & (3|4)"
    ),
    TestCase(
        "dist_complex_group",
        And(1, Or(And(2, 3), 4)),
        [[1, 2, 3], [1, 4]],
        "Distribute 1 over a mixed group ((2&3) | 4)"
    ),
    TestCase(
        "dist_complex_2",
        And(1, Or(And(2, 3), 4, Or(5, 6))),
        [[1, 2, 3], [1, 4], [1, 5], [1, 6]],
        "Distribute 1 over a more complex mixed group ((2&3) | 4 | (5|6))"
    )
]

# 4. DEEP NESTING & RECURSION
complex_tests = [
    TestCase(
        "deep_alternating",
        And(1, Or(2, And(3, Or(4, 5)))),
        # Logic step-through:
        # And(3, Or(4,5)) -> Or(3-4, 3-5)
        # Or(2, ...) -> Or(2, 3-4, 3-5)
        # And(1, ...) -> Or(1-2, 1-3-4, 1-3-5)
        [[1, 2], [1, 3, 4], [1, 3, 5]],
        "Alternating layers of AND/OR/AND/OR"
    ),
    TestCase(
        "deep_expansion",
        And(Or(1, 2), Or(3, And(4, Or(5, 6)))),
        # Logic: (1|2) & (3 | (4&5) | (4&6))
        # This is a 2x3 cartesian product -> 6 paths
        [[1, 3], [1, 4, 5], [1, 4, 6],
         [2, 3], [2, 4, 5], [2, 4, 6]],
        "Complex unbalanced Cartesian product"
    )
]

# 5. EDGE CASES
edge_tests = [
    TestCase(
        "edge_single_empty_nested",
        And(Or(1)), 
        [[1]],
        "Redundant wrapping And(Or(1))"
    ),
    TestCase(
        "edge_redundant_values",
        Or(And(1, 2), And(1, 2)),
        [[1, 2], [1, 2]], 
        "Identical paths (Sets vs Lists behavior)"
    )
]

# --- Master List ---
all_test_cases = (
    atomic_tests + 
    associativity_tests + 
    distributivity_tests + 
    complex_tests + 
    edge_tests
)

# Convert to simple list for looping if needed, or keep dictionaries
comprehensive_suite = [
    (t.id, asdict(t.input), t.expected) 
    for t in all_test_cases
]

In [ ]:
import itertools

def to_dnf(node):
    """
    Recursively flattens a logic tree of arbitrary depth into a 2D matrix.
    Returns: List[List[int]] (Disjunctive Normal Form)
    """
    # base: no conjunctions
    if not isinstance(node, dict):
        return [[node]] if node is not None else []

    # extract logic & children
    conj = node.get("conj")
    children = node.get("items")
    if not children:
        return []

    # base: And/Or depth=1
    if all(isinstance(child, int) for child in children):
        if conj == "And":
            return [children]  # And(1, 2) -> [[1, 2]]
        else: 
            return [[x] for x in children]  # Or(1, 2) -> [[1], [2]]

    # recurse children to child matrices
    child_matrices = [to_dnf(child) for child in children]

    # DNF algorithm: apply associative property on Or(1, 2, Or(3))
    if conj == "Or":
        merged_matrix = []
        for matrix in child_matrices:
            merged_matrix.extend(matrix)
        return merged_matrix

    # DNF algorithm: apply distributive property (And over Or)
    #    (A OR B) AND (C OR D)
    # => (A AND (C OR D)) OR (B AND (C OR D))
    # => (A AND C) OR (A AND D) OR (B AND C) OR (B AND D)
    elif conj == "And":
        product = itertools.product(*child_matrices)
        
        merged_matrix = []
        for combination in product:
            new_clause = []
            for clause in combination:
                new_clause.extend(clause)
            merged_matrix.append(new_clause)
            
        return merged_matrix
    
    return []



In [ ]:
for test_id, expr, mtx in comprehensive_suite:
    # print("Now testing:", test_id)
    result = to_dnf(expr)
    assert result == mtx, f"FAILED {test_id}: Expected {mtx}, got {result}"
    print("PASSED", test_id)

PASSED atomic_and
PASSED atomic_or
PASSED simple_and
PASSED base_simple_or
PASSED deep_atomic_and
PASSED deep_atomic_or
PASSED deep_atomic_mixed
PASSED assoc_nested_and
PASSED assoc_nested_or
PASSED assoc_mixed_depth
PASSED dist_simple
PASSED dist_right_side
PASSED dist_cartesian_product
PASSED dist_complex_group
PASSED dist_complex_2
PASSED deep_alternating
PASSED deep_expansion
PASSED edge_single_empty_nested
PASSED edge_redundant_values
